In [ ]:
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_cbc
import helpers_eda_inicial

Abrimos el archivo otorgado por el CBC y vemos el contenido de las paginas.

In [2]:
CBC = '../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx'

In [ ]:
# Load the data
xls = pd.ExcelFile(CBC)
for sheet_name in xls.sheet_names:
    print(sheet_name)

In [4]:
df_calificaciones = pd.read_excel(CBC, sheet_name='Calificaciones')
df_carreras = pd.read_excel(CBC, sheet_name='Materias Grilla Carreras')
df_convenciones = pd.read_excel(CBC, sheet_name='Convenciones')

Revisamos que todos los DNI sean de longitud 8.

In [ ]:
df_calificaciones['Dni'] = df_calificaciones['Dni'].astype(str)
# Largo de cada string
df_calificaciones['longitud'] = df_calificaciones['Dni'].str.len()
# Estadísticas descriptivas
print(df_calificaciones['longitud'].describe())
df_calificaciones['longitud'].value_counts().sort_index().plot(kind='bar')

In [ ]:
df_calificaciones[df_calificaciones['longitud'] != 8]['Dni'].unique()

Para hacer una primera revisión de datos.

df_calificaciones contiene las materias rendidas por estudiantes junto con sus clasificaciones.

In [ ]:
helpers_eda_inicial.dataset_profiling(df_calificaciones, 'df_calificaciones')

df_carreras enumera las materias del CBC que se realizan cen cada una de las carreras, junto con el codigo de la materia.

In [ ]:
df_carreras.head(10)

In [ ]:
helpers_eda_inicial.dataset_profiling(df_carreras, 'df_carreras')

No está pensado en formato tabla, explica el significado y contenido de algunas de las columnas:

In [ ]:
df_convenciones.head(10)

# EDA

Veo que algunas veces aparecen nombres en mayuscula y minuscula y que tienen espacios extra al comienzo o final, lo corregimos.

In [ ]:
df_carreras = helpers_cbc.normalize_column_values(df_carreras, ['Carrera', 'Materia'])
df_calificaciones = helpers_cbc.normalize_column_values(df_calificaciones, ['Carrera', 'Dirección', 'Localidad', 'dominio email', 'Materia', 'Nota', 'UBA XXI', 'Es materia FCEN?'])

Vemos que hay 145224 filas distintas pero en realidad se corresponde con 16229 DNIs distintos (personas distintas).

Además, figuran 98 dominios de mails distintos, pero si revisamos la distribución más del 85% se corresponde con gmail.

La materia que más figura es quimica.

Además, 'Dirección' y 'Localidad' con los que más valores faltantes presentan.

In [ ]:
helpers_eda_inicial.initial_eda(df_calificaciones, "df_calificaciones")

In [ ]:
# Variables a analizar
categoricas_principales = ['Carrera', 'dominio email', 'Materia', 'Nota', 'UBA XXI', 'Es materia FCEN?']

# Crear subplots con layout más generoso
fig, axes = plt.subplots(nrows=len(categoricas_principales), ncols=1, figsize=(18, 6 * len(categoricas_principales)), constrained_layout=True)

# Asegurar que axes sea iterable
if len(categoricas_principales) == 1:
    axes = [axes]

for i, col in enumerate(categoricas_principales):
    ax = axes[i]
    
    # Calcular distribución
    ciclo_counts = df_calificaciones[col].value_counts(normalize=True)
    
    # Plot
    ciclo_counts.plot(kind="bar", color="purple", ax=ax, width=0.8)

    # Ajustes visuales
    ax.set_title(f"Distribución de {col}", fontsize=14)
    ax.set_xlabel(col, fontsize=12)
    ax.set_ylabel("Proporción", fontsize=12)
    ax.tick_params(axis='x', rotation=70)  # Rotar etiquetas del eje x
    ax.tick_params(axis='y', labelsize=10)
    
    # Si hay muchas categorías, reducir el tamaño de fuente de las etiquetas del eje X
    if len(ciclo_counts) > 20:
        ax.set_xticklabels(ax.get_xticklabels(), fontsize=8)

plt.show()

## Carreras

In [ ]:
helpers_eda_inicial.initial_eda(df_carreras, "df_carreras")

Ambos dataset tienen 18 carreras distintas. Sin embargo, notamos que en df_calificaciones hay 32 materias distintas pero en df_carreras hay solo 10.

Analizando df_carreras vemos que todas tienen 7 o 6 materias asignadas para el CBC.

In [ ]:
materias_por_carrera = df_carreras.groupby("Carrera")["Materia"].nunique().reset_index()
materias_por_carrera.columns = ["Carrera", "Materias Únicas"]
materias_por_carrera = materias_por_carrera.sort_values(by="Materias Únicas", ascending=False)
print(materias_por_carrera)

Las materias FISICA, INT. AL CONOC. DE LA SOCIEDAD Y EL ESTADO, INT. AL PENSAMIENTO CIENTIFICO y QUIMICA, pertenecen a todas las carreras. Mientras que el resto pertenecen solo a un subconjunto.

In [ ]:
carreras_por_materia = df_carreras.groupby("Materia")["Carrera"].nunique().reset_index()
carreras_por_materia.columns = ["Materia", "Carreras Únicas"]
carreras_por_materia = carreras_por_materia.sort_values(by="Carreras Únicas", ascending=False)
print(carreras_por_materia)

Vemos en qué carreras aparecen aquellas materias que no pertenecen a todas las carreras.

In [ ]:
materias_con_pocas_carreras = carreras_por_materia[carreras_por_materia['Carreras Únicas'] < 18]

# Paso 2: filtrar el DataFrame original para quedarte solo con esas materias
materias_filtradas = df_carreras[df_carreras["Materia"].isin(materias_con_pocas_carreras['Materia'].unique())]

# Paso 3: agrupar para ver en qué carreras aparece cada materia
resultado = materias_filtradas.groupby("Materia")["Carrera"].unique().reset_index()

# Opcional: convertir la lista de carreras a string para mejor visualización
resultado["Carrera"] = resultado["Carrera"].apply(lambda x: ", ".join(x))

print(resultado.to_markdown(index=False))

Vemos en qué carreras no aparecen.

In [ ]:
# Paso 1: lista completa de carreras
todas_las_carreras = set(df_carreras["Carrera"].unique())

# Paso 2: contar en cuántas carreras aparece cada materia
materias_por_carrera_count = df_carreras.groupby("Materia")["Carrera"].nunique()

# Paso 3: filtrar materias que aparecen en menos de 18 carreras
materias_incompletas = materias_por_carrera_count[materias_por_carrera_count < 18].index

# Paso 4: obtener las carreras en las que NO aparece cada una de esas materias
faltantes = []
for materia in materias_incompletas:
    carreras_con_materia = set(df_carreras[df_carreras["Materia"] == materia]["Carrera"])
    carreras_sin_materia = todas_las_carreras - carreras_con_materia
    faltantes.append({
        "Materia": materia,
        "Carreras sin esa materia": ", ".join(sorted(carreras_sin_materia))
    })

# Convertir a DataFrame
faltantes_df = pd.DataFrame(faltantes)
print(faltantes_df.to_markdown(index=False))